<a href="https://colab.research.google.com/github/atheermoh/spn-linear-cryptanalysis/blob/main/spn_linear_attack_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Cryptanalysis of a Toy SPN Cipher

This notebook implements a linear cryptanalysis attack on a lightweight 8-bit Substitution–Permutation Network (SPN) cipher.

The goals are:
- To analyse biased linear relations involving the S-box,
- To use these relations to recover the **last-round key K₄**,
- To extend the attack backwards and recover **K₃** as well.



In [ ]:
import numpy as np

# -------------------------------------------------------------
#   S-BOX AND INVERSE S-BOX
# -------------------------------------------------------------

S_Box = [
    0xE, 0x4, 0xD, 0x1,
    0x2, 0xF, 0xB, 0x8,
    0x3, 0xA, 0x6, 0xC,
    0x5, 0x9, 0x0, 0x7
]

Inv_S_Box = [0] * 16
for i in range(16):
    Inv_S_Box[S_Box[i]] = i

# -------------------------------------------------------------
#   UTILITY FUNCTIONS
# -------------------------------------------------------------

def split_nibbles(value):
    return value >> 4, value & 0xF

def combine_nibbles(n1, n2):
    return (n1 << 4) | n2

def xor(a, b):
    return a ^ b

def apply_sbox_8bit(value):
    left, right = split_nibbles(value)
    return combine_nibbles(S_Box[left], S_Box[right])

def apply_inv_sbox_8bit(value):
    left, right = split_nibbles(value)
    return combine_nibbles(Inv_S_Box[left], Inv_S_Box[right])

# -------------------------------------------------------------
#   LOADING PLAINTEXT–CIPHERTEXT PAIRS
# -------------------------------------------------------------

def load_pairs(file_path):
    pairs = []
    with open(file_path, "r") as f:
        for line in f:
            p, c = map(int, line.strip().split())
            pairs.append((p, c))
    return pairs

# -------------------------------------------------------------
#   KEY RECOVERY FOR K4
# -------------------------------------------------------------

def reverseCK(ciphertext, K4):
    return xor(ciphertext, K4)

def test_approximation(p, c, K4):
    U3 = reverseCK(c, K4)
    U3_inv = apply_inv_sbox_8bit(U3)

    P3 = (p >> 3) & 1
    P5 = (p >> 5) & 1
    U3_2 = (U3_inv >> 2) & 1
    U3_6 = (U3_inv >> 6) & 1

    return (P3 ^ P5 ^ U3_2 ^ U3_6) == 0

def recover_K4(pairs):
    scores = []
    for K4 in range(256):
        count = sum(test_approximation(p, c, K4) for p, c in pairs)
        scores.append((K4, count))

    return sorted(scores, key=lambda x: x[1], reverse=True)

# -------------------------------------------------------------
#   KEY RECOVERY FOR K3
# -------------------------------------------------------------

def reverse_U2(ciphertext, K4, K3):
    U3 = reverseCK(ciphertext, K4)
    return apply_inv_sbox_8bit(U3 ^ K3)

def test_approximation_K3(p, c, K4, K3):
    U2 = reverse_U2(c, K4, K3)

    P3 = (p >> 3) & 1
    P5 = (p >> 5) & 1
    U2_2 = (U2 >> 2) & 1
    U2_6 = (U2 >> 6) & 1

    return (P3 ^ P5 ^ U2_2 ^ U2_6) == 0

def recover_K3(pairs, K4):
    scores = []
    for K3 in range(256):
        count = sum(test_approximation_K3(p, c, K4, K3) for p, c in pairs)
        scores.append((K3, count))

    return sorted(scores, key=lambda x: x[1], reverse=True)

# -------------------------------------------------------------
#   MAIN EXECUTION PIPELINE
# -------------------------------------------------------------

def main():
    file_path = "data/Encryptions256-2024-25.txt"
    pairs = load_pairs(file_path)

    print("Recovering K4...")
    k4_results = recover_K4(pairs)
    best_K4, score4 = k4_results[0]
    print(f"Best K4 = {best_K4:08b} (score {score4}/256)\n")

    print("Recovering K3...")
    k3_results = recover_K3(pairs, best_K4)
    best_K3, score3 = k3_results[0]
    print(f"Best K3 = {best_K3:08b} (score {score3}/256)\n")

if __name__ == "__main__":
    main()
